# Building a Better Title-Caser, Part 1: Beyond Python str.title

Title-casing text is one of those hard problems no one ever gets right, yet no one considers worthy enough to solve with AI. Here I experiment to see if I can improve upon the latest best solutions with a local Ollama modelfile and a solid prompt.

## Setup

In [5]:
import google.generativeai as genai
import ollama
from titlecase import titlecase

## Built-In `title`

I begin by seeing what `str.title` does. It's built into Python, so nothing needs to be installed. 

In [6]:
"hi".title()

'Hi'

## PyPI `titlecase`

Now with `pip install titlecase`, I get this `titlecase` function:

In [7]:
titlecase("hi")

'Hi'

## Simple Multi-Word Ccomparison

Both of these functions should do well with a simple test case:

In [8]:
text = "the quick brown fox"
print(f"title():    {text.title()}")
print(f"titlecase(): {titlecase(text)}")

title():    The Quick Brown Fox
titlecase(): The Quick Brown Fox


## With Apostrophes

In [9]:
## With apostrophes
text2 = "it's a beautiful day in mr. rogers' neighborhood"
print(f"\ntitle():    {text2.title()}")
print(f"titlecase(): {titlecase(text2)}")


title():    It'S A Beautiful Day In Mr. Rogers' Neighborhood
titlecase(): It's a Beautiful Day in Mr. Rogers' Neighborhood


Here `titlecase` lowercased the articles correctly.

## Modern Terms With Unconventional Capitalization 

In [10]:
text3 = "iphone and e-mail tips for pdfs"
print(f"\ntitle():    {text3.title()}")
print(f"titlecase(): {titlecase(text3)}")


title():    Iphone And E-Mail Tips For Pdfs
titlecase(): Iphone and E-Mail Tips for PDFS


My use case would be to title case voice-dictated text. Here there's something tricky because E-Mail is one of those terms where the hyphenation is debatable and undergoing change. Personally, I prefer email without the hyphen. It's interesting how I voice-dicated this paragraph (Wispr Flow) and it ended up both ways! 

My preference for a return value here is `iPhone and Email Tips for PDFs`. In situations where a hyphenated word is optionally unhyphenated, I'd like the title-casing function to unhyphenate and then title-case it. If that's not possible, my backup preference is `iPhone and E-Mail Tips for PDFs`.

## Using a Hosted LLM as a Title-Caser

In [11]:
def tc_gemini(s):
    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    resp = model.generate_content(f"Convert '{s}' to title case, please. Return ONLY the title-cased string.", safety_settings=[], request_options={"timeout": 1000})
    try:
        return resp.text
    except Exception as ex:
        raise ex

In [12]:
tc_gemini(text3)

'iPhone and E-mail Tips for PDFs\n'

Gemini 1.5 Flash works decently as a title caster with this simple prompt. I noticed though that the mail and email isn't capitalized. That is one that people find confusing. The rule is when a word is hyphenated, each part of the hyphenated word should be capitalized. 

This feels a bit wasteful though with a lot of API calls to a service that will likely cost money in the future. I suppose you'd want to batch them if you went this way. I think it would be a lot nicer though to use a small local LLM for simple tasks like this. 

## Use Small Local LLMs as Title-Casers

In [14]:
def tc_ollama(s, model='mistral'):
    # Call ollama with a simple title-case prompt
    response = ollama.chat(model=model, messages=[{
        'role': 'user',
        'content': f"Convert '{s}' to title case. Return ONLY the title-cased string with no explanation or quotes."
    }])
    return response['message']['content'].strip()

In [15]:
print(tc_ollama(text3))

[2025-02-16 07:31:46 - httpx:1025 - INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


iPhone and Email Tips for PDFs


Mistral is quite good. Let's try others:

In [16]:
# Let's try a few different models to compare
models = ['llama3.2', 'tinyllama', 'deepseek-r1:7b', 'deepseek-coder:33b', 'qwen2.5:3b']
print("\nComparing models:")
for model in models:
    try:
        print(f"{model:10}: {tc_ollama(text3, model)}")
    except:
        print(f"{model:10}: Failed")


Comparing models:


[2025-02-16 07:32:14 - httpx:1025 - INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


llama3.2  : Iphone And E-Mail Tips For PDFs


[2025-02-16 07:32:23 - httpx:1025 - INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


tinyllama : "IPHONE AND EMAIL TIPS FOR PDFS"


...

deepseek-r1:7b: <think>
Okay, so I need to convert "ipone and e-mail tips for pdfs" into title case. Hmm, first, what's title case? From what I remember, it's when each major word in a sentence is capitalized. So every time there's a new word that starts with a letter, that letter should be uppercase.

Let me break down the original string: "ipone and e-mail tips for pdfs". Wait, let's make sure to split this correctly into words because sometimes it's easy to miscount or miss the spaces. I think the correct version is "iPhone and E-Mail Tips for PDFs". 

Wait a second, no. The user wrote "ipone" but maybe that was a typo? iPhone should be "iPhone", right? So "ipone" might actually be incorrect. But in the given string, it's written as "ipone and e-mail tips for pdfs". Maybe they meant to say "iphone", so I'll proceed with that.

So the original words are: "ipone", "and", "e-mail", "tips", "for", "pdfs".

Now applying title case step by step:

1. The first word is "ipone" – in title ca

[2025-02-16 07:35:25 - httpx:1025 - INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


deepseek-coder:33b: Here is a Python solution that uses built in python methods to convert a sentence into title case.

```python
def convert_to_titlecase(sentence):
    return sentence.title()

print(convert_to_titlecase('iphone and e-mail tips for pdfs'))
```
When you run this code, it will print: `'Iphone And E-Mail Tips For Pdfs'`


[2025-02-16 07:35:28 - httpx:1025 - INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


qwen2.5:3b: Iphone And Email Tips For Pdfs
